<a id="top"></a>
# MICCAI 2017 Robotic Instrument Segmentation


## OpenVINO version check:
You are currently using the latest development version of Intel® Distribution of OpenVINO™ Toolkit. Alternatively, you can open a version of this notebook for the Intel® Distribution of OpenVINO™ Toolkit LTS version by running the cell below and following the link it generates.

In [ ]:
from qarpo import displayMultiversionURL
import os
displayMultiversionURL(os.path.abspath(""), "robotic-surgery-segmentation.ipynb", "openvino-dev-latest", ['openvino-lts'])

## Prerequisites
This sample requires the following:
- All files are present and in the following directory structure:
    
    - **robotic-surgery-segmentation.ipynb** - This Jupyter* Notebook
    - **figues/segmentation.gif,TernausNet.png** - Images that appear in this Jupyter* Notebook
    - **python/\*** - Python* code for all aspects of the robotic instrument segmentation application:<br>
    -- **figures.py** - Generates visualization for steps of segmentation performed<br>
    -- **models.py** - Contains different models as Python* code<br>
    -- **pytorch_to_onnx.py** - Converts PyTorch* to ONNX* model<br>
    -- **img_to_video.py** - Converts multiple images into a video file<br>
    -- **pytorch_infer.py** - Runs inference using PyTorch\*<br>
    -- **segmentation_parts.py** - Robotic instrument segmentation application code<br>
    -- **utils.py** - Utility functions<br>
    -- **run_py.sh** - Utility shell script to run Python* files from jobs<br>
    -- **quantize.py** - Converts OpenVINO FP32 to INT8 model<br>
    - **data/short_source.mp4** - Test video
    - **data/frame.png,left_framge.png,right_frame.png** - Used for visualization 
    - **/data/robotic-instrument-segmentation/unet11_\*_20/model_0.pt** - Trained U-Net model data for different approaches (binary, parts, instruments - "parts" will be used)
    

It is recommended that you have already read the following from [Get Started on the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/home/):
- [Overview of the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/get_started/devcloud/)
- [Overview of the Intel® Distribution of OpenVINO™ toolkit](https://devcloud.intel.com/edge/get_started/openvino/)

<br><div class=note><i><b>Note: </b>It is assumed that the server this sample is being run on is on the Intel® DevCloud for the Edge which has Jupyter* Notebook customizations and all the required libraries already installed.  If you download or copy to a new server, this sample may not run.</i></div>


## Introduction

This sample application demonstrates how a smart video IoT solution may be created using Intel® hardware and software tools to perform robotic instrument segmentation.  This solution performs semantic segmentation to identify the segments of robotic instruments within a video frame.  The identified robotic instruments segments are then highlighted in the output with each segment appearing in a different color.

### Key concepts
This sample application includes an example for the following:
- Application:
  - Video and image input is supported using OpenCV
  - OpenCV is used to draw bounding boxes around detected objects, labels, and other information
  - Visualization of the resulting segmentation in the output
- Intel® DevCloud for the Edge:
  - Submitting inference as jobs that are performed on different edge compute nodes (rather than on the development node hosting this Jupyter* notebook)
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit):
  - Create the necessary Intermediate Representation (IR) files for the inference model using [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
  - Run an inference application on multiple hardware devices using the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)


### Application background
![Robotic Instrument Challenge](./figures/segmentation.gif)

The code in this sample refers to the winning solution by Alexey Shvets, Alexander Rakhlin, Alexandr A. Kalinin, and Vladimir Iglovikov in the [MICCAI 2017 Robotic Instrument Segmentation Challenge](https://endovissub2017-roboticinstrumentsegmentation.grand-challenge.org/). This Jupyter* Notebook has been modified from the original found on [GitHub](https://github.com/ternaus/robot-surgery-segmentation/blob/master/Demo.ipynb) which is made available with an [MIT license](https://github.com/ternaus/robot-surgery-segmentation/blob/master/LICENSE). The data files necessary to run this notebook are included in `/data/robotic-surgery-segmentation`.

![TernausNet](./figures/TernausNet.png)

## Robotic instrument segmentation application
The robotic instrument segmentation application uses the Intel® Distribution of OpenVINO™ toolkit to perform inference on an input video to locate robotic instruments within each frame.  We will setup, run, and view the results for this application for several different hardware devices (CPU. GPU, etc.) available on the compute nodes within the Intel® DevCloud for the Edge.  To accomplish this, we will be performing the following tasks:

1. Use the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) to create the inference model IR files needed to perform inference
2. Create the job file used to submit running inference on compute nodes
3. Submit jobs for different compute nodes and monitor the job status until complete
4. View results and assess performance 

### How it works
At startup the robotic instrument segmentation application configures itself by parsing the command line arguments.  Once configured, the application loads the specified inference model's IR files into the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) and runs inference on the specified input video to identify segments of robotic instruments.  Once identified, each robotic instrument segment is colored to highlight it on the output video.

To run the application on the Intel® DevCloud for the Edge, a job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU and Intel® Movidius™ Neural Compute Stick 2.  After inference on the input is completed, the output is stored in the appropriate `results/<architecture>/` directory.  The results are then viewed within this Jupyter* Notebook using the `videoHTML` video playback utility.

The application and inference code for this sample is already implemented in the Python* file [`segmentation_parts.py`](./python/segmentation_parts.py) (and other helper `python/*.py` files).

The following sections will guide you through configuring and running the robotic instrument segmentation application.

### Configuration
The following sections describe all the necessary configuration to run the robotic instrument segmentation application.
#### Command line arguments
The application is run from the command line using the following format:
```bash
python3 python/segmentation_parts.py <arguments...>
```
The required command line _<arguments...>_ to run the Python* executable [`segmentation_parts.py`](./python/segmentation_parts.py) are:
- **-m** - The precision to complete the path to the _.xml_ IR file (created using [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)) for the inference model. The file is found using the path `models/ov/<`*precision*`>/surgical_tools_parts.xml`
- **-i** - Path to input video file
- **-o** - The path to where the output video file will be stored
- **-d** - Device type to use to run inference (CPU, GPU, MYRIAD, CPU)

### Imports
We begin by importing all the Python* modules that will be used within this Jupyter* Notebook to run and display the results of the robotic instrument segmentation application on the Intel® DevCloud for the Edge:
- [cv2](https://opencv.org/) - Python* OpenCV module
- [os](https://docs.python.org/3/library/os.html#module-os) - Operating system specific module (used for file name parsing)
- [time](https://docs.python.org/3/library/time.html#module-time) - Time tracking module (used for measuring execution time)
- [matplotlib.pyplot](https://matplotlib.org/) - pyplot is used for displaying output images
- [sys](https://docs.python.org/3/library/sys.html#module-sys) - System specific parameters and functions
- [qarpo.demoutils](https://github.com/ColfaxResearch/qarpo) - Provides utilities for displaying results and managing jobs from within this Jupyter* Notebook
- [python.utils](./python/utils.py) - Provides utilities for displaying results within this Jupyter* Notebook

Run the following cell to import the Python* dependencies needed.

<br><div class=tip><b>Tip: </b>Select a cell and then use **Ctrl+Enter** to run that cell.</div>

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import time
import sys
from qarpo.demoutils import *
from python.utils import create_script, mask_overlay
print('Imported Python modules successfully.')

## Inference with PyTorch (Optional)
Run the following cell to perform inference with the PyTorch model on a compute node with an [Intel Core i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-) using the code in [pytorch_infer.py](python/pytorch_infer.py). 

<br><div class=note><i><b>Note: </b>The next cell uses `qsub` to submit a job to be run on a compute node because the login node running this Jupyter* Notebook does not have enough memory to run the task.</i></div>

In [ ]:
# Run the script
job_id_infer = !qsub python/run_py.sh -l nodes=1:idc001skl -F "python/pytorch_infer.py" -N pytorch_core
if job_id_infer:
    print(job_id_infer[0])
    progressIndicator('results/', job_id_infer[0]+'.txt', "Inference", 0, 100)

<br><div class=danger><b>Wait!: </b>
Please wait for the progress bar and job to complete before proceeding to the next cell.
</div>

To view the original input image and the inference results, run the following cell

In [ ]:
# read, label, and display original input image
image = cv2.imread("generated/input.png")
plt.figure(1, figsize=(15, 15))
plt.subplot(121)
plt.axis('off')
plt.title("Input Image")
plt.imshow(image)

# read, lable, and display result image
mask = cv2.imread("generated/mask.png")
plt.subplot(122)
plt.axis('off')
plt.title("Segmentation")
plt.imshow(mask_overlay(image, mask));

## Converting PyTorch* to ONNX*

The ONNX* models need to be generated from the original PyTorch* models to be used with the Intel® Distribution of OpenVINO™ toolkit.  This is done by running [pytorch_to_onnx.py](python/pytorch_to_onnx.py). 

<br><div class=note><i><b>Note: </b>The next cell uses `qsub` to submit a job to be run on a compute node because the login node running this Jupyter* Notebook does not have enough memory to run the task.</i></div>

In [ ]:
# Run the script
job_id_onnx = !qsub python/run_py.sh -l nodes=1:idc001skl -F "python/pytorch_to_onnx.py" -N pytorch_to_onnx
if job_id_onnx:
    print(job_id_onnx[0])
    progressIndicator('results/', job_id_onnx[0]+'.txt', "Conversion", 0, 100)

<br><div class=danger><b>Wait!: </b>
Please wait for the progress bar and job to complete before proceeding to the next cell.
</div>

The two ONNX* models that have now be generated are:
- **models/onnx/surgical_tools_parts.onnx** - Segmentation model used for identifying robotic instrument segments ("parts")
 - This model will be used to process video and measure performance in later steps 
- **models/onnx/surgical_tools.onnx** - Segmentation model used for identifying robotic instrument ("binary")
 - This model, along with the `surgical_tools_parts.onnx`, will be used in an extra inference run to show some visualization of the segmentation involved.

### Create the IR files for the inference model

The Intel® Distribution of OpenVINO™ toolkit includes the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) used to convert and optimize trained models into the Intermediate Representation (IR) model files, and the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) that uses the IR model files to run inference on hardware devices.  The IR model files can be created from trained models from popular frameworks (e.g. Caffe\*, Tensorflow*, etc.). For this sample, the two ONNX* models that will be used were just generated from PyTorch* in the previous step.



The `surgical_tools_parts.onnx` and `surgical_tools.onnx` files will need to be optimized using the Model Optimizer to create the necessary IR files.  We will be running the inference model on different hardware devices which have different requirements on the precision of the model (see [Inference Engine Supported Model Formats](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_Supported_Devices.html#supported_model_formats) for details).  For our purposes, we will focus on the use of the most common precision `FP16` to run on all devices that we target.

For this model, we will run the Model Optimizer using the format:
```bash
mo.py \
    --input_model <path_to_model> \
    --output_dir <path_to_output_directory> \
    --data_type <data_precision> \
    --move_to_preprocess \
    --scale_values [<scale_values>] \
    --mean_values [<channel_mean_values>]
```

The input arguments are as follows:
- **--input_model** : The model's input *.pb* file
- **--output_dir** : Output directory where to store the generated IR model files
- **--data_type** : The model's data type and precision (e.g. FP16, FP32, INT8, etc.)
- **--move_to_preprocess** : Move mean values to IR preprocess section
- **--scale_values** : Scaling (divide by, one per channel) value to apply to input values
- **--mean_values** : Mean values (one per channel) to be subtracted from input values before scaling

The input shape, scaling values, and mean values we will be using are specific to the model topology being used.  Using the appropriate values for the model that we will use, The complete command will look like the following:
```bash
!mo.py \
    --input_model <path_to_model> \
    --data_type <data_precision> \
    --output_dir models/<data_precision> \
    --move_to_preprocess \
    --scale_values [58.395,57.12,57.375] \
    --mean_values [123.75,116.28,103.58]
```
We will run the command once on each model with <*path_to_model*> set to `models/onnx/surgical_tools.onnx` and `models/onnx/surgical_tools_parts.onnx` and with <*data_precision*> set to `FP16`.  This may be changed (e.g. `FP32`, etc.) as needed to run inference on other different devices.

<br><div class=note><i><b>Note: </b>More information on how to use Model Optimizer to convert ONNX* models as well as specifying input shape and scaling parameters for common model topologies may be found at:[Converting a ONNX* Model](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_ONNX.html)
</i></div>


Run the following cell to use the Model Optimizer to create the model IR files:

In [ ]:
!mo_onnx.py \
    --input_model "models/onnx/surgical_tools.onnx" \
    --output_dir models/ov/FP16/ \
    --data_type FP16 \
    --move_to_preprocess \
    --scale_values "[0.229, 0.224, 0.225]" \
    --mean_values "[0.485, 0.456, 0.406]"

!mo_onnx.py \
    --input_model "models/onnx/surgical_tools_parts.onnx" \
    --output_dir models/ov/FP16/ \
    --data_type FP16 \
    --move_to_preprocess \
    --scale_values "[0.229, 0.224, 0.225]" \
    --mean_values "[0.485, 0.456, 0.406]"

!echo "\nAll IR files that were created:"
!find ./models -name "*.xml" -o -name "*.bin"

<br><div class=tip><i><b>Tip: </b>The '!' at the beginning of a line is a special Jupyter* Notebook command that allows you to run shell commands as if you are at a command line. The above command will also work in a terminal (with the '!' removed).</i></div>

As shown above from the output of the last `!find...` command, the required sets of IR model files (`*.xml` and `*.bin`) have been created.  

### Run inference
The following sections will go through the steps to run our inference application on the Intel® DevCloud for the Edge. 

#### Configure input
For convenience and consistency, in the next cell we set the Python* variable `InputVideo` to the input video file we will be using to run our sample application.

<br><div class=note><i><b>Note: </b>
If you want to use a different input video, change the path in the following cell to the path of the video and run the cell again.
</i></div>

In [ ]:
# Set the path to the input video to use for the rest of this sample
InputVideo = "data/short_source.mp4"
print(f"Input video file set to:{InputVideo}")

#### Create the job file
We will run inference on several different edge compute nodes present in the Intel® DevCloud for the Edge. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file is a [Bash](https://www.gnu.org/software/bash/) script that serves as a wrapper around the Python* executable of our application that will be executed directly on the edge compute node.  One purpose of the job file is to simplify running an application on different compute nodes by accepting a few arguments and then performing accordingly any necessary steps before and after running the application executable.  

For this sample, the job file we will be using is already written for you and appears in the next cell.  The job file will be submitted as if it were run from the command line using the following format:
```bash
segmentation_job.sh <output_directory> <device> <fp_precision> <input_file> 
```
Where the job file input arguments are:
- <*output_directory*> - Output directory to use to store output files
- <*device*> - Hardware device to use (e.g. CPU, GPU, etc.)
- <*fp_precision*> - Which floating point precision inference model to use (FP32 or FP16)
- <*input_file*> - Path to input video file

Based on the input arguments, the job file will do the following:
- Change to the working directory `PBS_O_WORKDIR` where this Jupyter* Notebook and other files appear on the compute node
- Create the <*output_directory*>
- Choose the appropriate inference model IR file for the specified <*fp_precision*>
- Run the application Python* executable with the appropriate command line arguments

Run the following cell to create the `segmentation_job.sh` job file.  The [`%%writefile`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) line at the top will write the cell contents to the specified job file `segmentation_job.sh`.

In [ ]:
%%writefile segmentation_job.sh

# Store input arguments: <output_directory> <device> <fp_precision> <input_file>
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
INPUT_FILE=$4

# The default path for the job is the user's home directory,
#  change directory to where the files are.
cd $PBS_O_WORKDIR

# Make sure that the output directory exists.
mkdir -p $OUTPUT_FILE

# Run the robotic instrument segmentation code
python3 python/segmentation_parts.py  -m $FP_MODEL \
                                      -i $INPUT_FILE \
                                      -d $DEVICE \
                                      -o $OUTPUT_FILE


#### How to submit a job

Now that we have the job script, we can submit jobs to edge compute nodes in the Intel® DevCloud for the Edge.  To submit a job, the `qsub` command is used with the following format:
```bash
qsub <job_file> -N <JobName> -l <nodes> -F "<job_file_arguments>" 
```
We can submit segmentation_job.sh to several different types of edge compute nodes simultaneously or just one node at a time.

There are three options of `qsub` command that we use for this:
- <*job_file*> - This is the job file we created in the previous step
- `-N` <*JobName*> : Sets name specific to the job so that it is easier to distinguish  between it and other jobs
- `-l` <*nodes*> - Specifies the number and the type of nodes using the format *nodes*=<*node_count*>:<*property*>[:<*property*>...]
- `-F` "<*job_file_arguments*>" - String containing the input arguments described in the previous step to use when running the job file

*(Optional)*: To see the available types of nodes on the Intel® DevCloud for the Edge, run the following cell.

In [ ]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

In the above output from executing the previous cell, the properties describe the node, and the number on the left is the number of available nodes of that architecture.

#### Submit jobs

Each of the cells in the subsections below will submit a job to be run on different edge compute nodes. The output of each cell is the _JobID_ for the submitted job.  The _JobID_ can be used to track the status of the job.  After submission, a job will go into a waiting queue before running once the requested compute nodes become available.

<br><div class=note><i><b>Note: </b>You may submit all jobs at once or one at a time.</i></div> 
<br><div class=note><i><b>Note: </b>Presently the model is not fully supported for VPU so we will not be running it on a Intel® Neural Compute Stick 2 nor a Intel® Vision Accelerator Design with Intel® Movidius™ VPUs.  There is an issue that prevents running on an Intel® Atom® and using the integrated Intel® GPU, so we will not be running on it too.</i></div>
<br><div class=tip><b>Tip: </b>**Shift+Enter** will run the cell and automatically move you to the next cell. This allows you to use **Shift+Enter** multiple times to quickly run through multiple cells, including markdown cells.</div>

##### Submit to an edge compute node with an Intel® CPU
In the cell below, we submit a job to an edge node with an [Intel® Core™ i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html) processor. The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_core = !qsub segmentation_job.sh -l nodes=1:idc001skl -F "results/core/ CPU FP16 {InputVideo}" -N robo_seg_core
print(job_id_core[0])
#Progress indicators
if job_id_core:
    progressIndicator('results/core',  f'i_progress_{job_id_core[0]}.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Xeon® Gold 6258R CPU
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Gold 6258R Processor](https://ark.intel.com/content/www/us/en/ark/products/199350/intel-xeon-gold-6258r-processor-38-5m-cache-2-70-ghz.html). The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon_cascade_lake = !qsub segmentation_job.sh -l nodes=1:idc018 -F "results/xeon_cascade_lake/ CPU FP16 {InputVideo}" -N robo_seg_xeon_cascade_lake
print(job_id_xeon_cascade_lake[0])
#Progress indicators
if job_id_xeon_cascade_lake:
    progressIndicator('results/xeon_cascade_lake',  f'i_progress_{job_id_xeon_cascade_lake[0]}.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Xeon® E3-1268L v5 CPU
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Processor E3-1268L v5](https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz.html). The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon_skylake = !qsub segmentation_job.sh  -l nodes=1:idc007xv5 -F "results/xeon_skylake/ CPU FP16 {InputVideo}" -N robo_seg_xeon_skylake 
print(job_id_xeon_skylake[0])
#Progress indicators
if job_id_xeon_skylake:
    progressIndicator('results/xeon_skylake', f'i_progress_{job_id_xeon_skylake[0]}.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Core CPU and using the integrated Intel® GPU
In the cell below, we submit a job to an edge node with an [Intel® Core i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html). The inference workload will run on the Intel® HD Graphics 530 GPU integrated with the CPU.

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub segmentation_job.sh -l nodes=1:idc001skl -F "results/gpu/ GPU FP16 {InputVideo}" -N robo_seg_gpu 
print(job_id_gpu[0])
#Progress indicators
if job_id_gpu:
    progressIndicator('results/gpu', f'i_progress_{job_id_gpu[0]}.txt', "Inference", 0, 100)

##### Generate example visualizations using an Intel Core CPU
In the cell below, we submit a job to run the script [figures.py](python/figures.py) on an edge node with an [Intel Core i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-). The workload will be run on the CPU.

In [ ]:
# Run the script
job_id_figures = !qsub python/run_py.sh -l nodes=1:idc001skl -F "python/figures.py" -N robo_seg_figures 
if job_id_figures:
    print(job_id_figures[0])
    progressIndicator('results/', job_id_figures[0]+'.txt', "Visualization", 0, 100)

### Monitor job status

To check the status of the jobs that have been submitted, use the `qstat` command.  The custom Jupyter* Notebook widget `liveQstat()` is provided to display the output of `qstat` with live updates.  

Run the following cell to display the current job status with periodic updates. 

In [ ]:
liveQstat()

You should see the jobs that you have submitted (referenced by the `JobID` that gets displayed right after you submit the jobs in the previous step).
There should also be an extra job in the queue named `jupyterhub-singleuser`: this job is your current Jupyter* Notebook session which is always running.

The `S` column shows the current status of each job: 
- If the status is `Q`, then the job is queued and waiting for available resources
- If ste status is `R`, then the job is running
- If the job is no longer listed, then the job has completed

<br><div class=note><i><b>
Note: The amount of time spent in the queue depends on the number of users accessing the requested compute nodes. Once the jobs for this sample application begin to run, they should take from 1 to 5 minutes each to complete.
</b></i></div>

<br><div class=danger><b>Wait!: </b>Please wait for the inference jobs and video rendering to complete before proceeding to the next step to view results.</div>

### View results

Once the jobs have completed, the queue system outputs the stdout and stderr streams of each job into files with names of the forms <*JobName*>.o<*JobID*> and <*JobName*>.e<*JobID*>, respecitvely.  The *JobName* corresponds to the `-N` option when submitting the job using the `qsub` command.  

The output video file for each job is written to the file `output.mp4` located in the directory `results/<device>` that was specified as the output directory to the job file.  We will now use the `videoHTML()` utility to display the output video files within this Jupyter* notebook.  Calling `videoHTML()` from a Python* cell follows the form:
```python
videoHTML(title, [list_of_video_files], statistics(optional))
```
The parameters are:
- *title* - Title to put at the top of the displayed output
- \[*list_of_video_files*\] - Python* list of video files to display
- *statistics(optional)* - Optional statistics file containing the number of seconds it took to process a number of frames

Run the cells below to display the videos.



##### View results from an Intel® CPU

In [ ]:
videoHTML('Intel Core CPU', 
          [f'results/core/output_{job_id_core[0]}.mp4'],f'results/core/stats_{job_id_core[0]}.txt')

##### View results from an Intel® Xeon® Gold 6258R CPU

In [ ]:
videoHTML('Intel® Xeon® Gold 6258R CPU', 
          [f'results/xeon_cascade_lake/output_{job_id_xeon_cascade_lake[0]}.mp4'],f'results/xeon_cascade_lake/stats_{job_id_xeon_cascade_lake[0]}.txt')

##### View results from an Intel® Xeon® E3-1268L v5 CPU

In [ ]:
videoHTML('Intel® Xeon® E3-1268L v5 CPU',
          [f'results/xeon_skylake/output_{job_id_xeon_skylake[0]}.mp4'],f'results/xeon_skylake/stats_{job_id_xeon_skylake[0]}.txt')

##### View results from an Intel® Core CPU and integrated Intel® GPU

In [ ]:
videoHTML('Intel Core + Intel GPU',
          [f'results/gpu/output_{job_id_gpu[0]}.mp4'],f'results/gpu/stats_{job_id_gpu[0]}.txt')

##### Visualize results on the  Intel® CPU

In [ ]:
# Display image
from IPython.display import Image
display(Image(filename='generated/predictions.png'))

### View and assess performance results

The running time of each inference task is recorded in `results/<device>/stats.txt`. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance for **Inference Engine Processing Time** and higher values mean better performance for **Inference Engine FPS**. When comparing results, please keep in mind that some architectures are optimized for highest performance, others for low power or other metrics.

In [ ]:
# Create table of <architecture>, <title> for plotting
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon_cascade_lake', 'Intel Xeon\nGold\n 6258R\nCPU'),
             ('xeon_skylake', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nHD530/GPU')]
# For each archtecture in table, create path to stats file or placeholder 
stats_list = []
for arch, a_name in arch_list:
    # if job_id_<architecture> exists, the job was run and has a stats file
    if 'job_id_'+arch in vars():
        stats_list.append((f'results/{arch}/stats_{vars()["job_id_"+arch][0]}.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))
# Plot the execution time from the stats files
summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time')
# Plot the frames per second from the stats files
summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps')

## Telemetry Dashboard
Once your submitted jobs are completed, run the cells below to view telemetry dashboards containing performance metrics for your model and target architecture.


In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view telemetry dashboard of the last job ran on Intel® Core™ i5-6500TE</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_core[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Xeon® Gold 6258R CPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_xeon_cascade_lake[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Xeon® E3-1268L v5 CPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_xeon_skylake[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view telemetry dashboard of the last job ran on Intel® Core CPU and using the integrated Intel® GPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_gpu[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

## Quantization

Quantization is the process of reducing the model's precision. By performing this optimization, you can accelerate your model execution time.  

[quantize.py](python/quantize.py) file contains quantization script and uses the [post training optimization toolkit (POT) API](https://docs.openvinotoolkit.org/latest/pot_compression_api_README.html) to reduce model's precision from FP32 to INT8. Quantization settings, such as the path to original model, path to dataset, quantization algorithm etc., which should be set via configs. ```DatasetsDataLoader``` creates quantization dataset from the sample video and loads one by one input images to POT, when quantization process starts. When quantization is finished, the INT8 model will be saved at ```'/models/ov/INT8'``` directory.

Run the following cell to create the ```quantization_job.sh``` job file. This script runs quantization and benchmarking of the quantized and non-quantized models to compare their execution time.

In [ ]:
%%writefile quantization_job.sh
export WORK_DIR=$PBS_O_WORKDIR

# Run the quantization script
python3 ${WORK_DIR}/python/quantize.py

# Run the benchmark_app for FP32 model
python3 /opt/intel/openvino/deployment_tools/tools/benchmark_tool/benchmark_app.py \
        -m ${WORK_DIR}/models/ov/FP16/surgical_tools_parts.xml 2>/dev/null | grep Throughput | xargs echo FP32

# Run the benchmark_app for INT8 model
python3 /opt/intel/openvino/deployment_tools/tools/benchmark_tool/benchmark_app.py \
        -m ${WORK_DIR}/models/ov/INT8/surgical_tools_parts.xml 2>/dev/null | grep Throughput | xargs echo INT8

Run the following cell to start quantization and benchmarking of the quantized model on Xeon Gold 6258R.

In [1]:
def wait_for_job_to_finish(job_id):
    if job_id:
        print("Job submitted to the queue. Waiting for it to complete: " + job_id[0])    
        
        filename = os.getcwd() + "/quantization_job.{}{}"
        
        filename_o = filename.format('o',job_id[0].split(".")[0])
        filename_e = filename.format('e',job_id[0].split(".")[0])
            
        # Wait until the job is ended and the report file is created.
        while not os.path.exists(filename_o) and not os.path.exists(filename_e):  
            time.sleep(1) 
            
        # Reading the benchmark_app results
        print('Results for non-optimized and optimized models:')
        with open(filename_o, 'r') as read_obj:
            for line in read_obj:            
                if 'Throughput:' in line:
                    print(line.split('\n')[0])
                      
    else:
        print("Error in job submission.")

job_id_core = get_ipython().getoutput('qsub quantization_job.sh -N quantization_job -l nodes=1:gold6258r')
benchmarks = wait_for_job_to_finish(job_id_core)

Job submitted to the queue. Waiting for it to complete: 68266.v-qsvr-1.devcloud-edge
Results for non-optimized and optimized models:
FP32 Throughput: 2.57 FPS
INT8 Throughput: 6.07 FPS


<br><div class=danger><b>Wait! </b>Please wait for the inference quantization job to complete.</div>

In this demo only 20 images were used for the model's quantization from FP32 to INT8. 
To achieve better accuracy, more data should be processed in quantization process to collect more robust statistics.

## Next steps
- [More Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) - sample application Jupyter* Notebook tutorials
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


## About this notebook

For technical support, please see the [Intel® DevCloud Forums](https://software.intel.com/en-us/forums/intel-devcloud-for-edge)

## Citation
    @inproceedings{shvets2018automatic,
    title={Automatic Instrument Segmentation in Robot-Assisted Surgery using Deep Learning},
    author={Shvets, Alexey A and Rakhlin, Alexander and Kalinin, Alexandr A and Iglovikov, Vladimir I},
    booktitle={2018 17th IEEE International Conference on Machine Learning and Applications (ICMLA)},
    pages={624--628},
    year={2018}
    }
<br><div class=note><i><b>Note: </b>Citation appears exactly as it appears on source website</i></div>

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>